In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
import wandb
project_name = "object_detection"
wandb.init(project=project_name, reinit=True)

wandb.run.name = 'baseline.ipynb'
# generted run ID로 하고 싶다면 다음과 같이 쓴다.
# wandb.run.name = wandb.run.id
wandb.run.save()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jongeson (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [3]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('./configs/trash/faster_rcnn/faster_rcnn_r50_fpn_1x_trash.py')
PREFIX = '../../input/data/'


In [4]:
print(cfg.dump())

model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='L1Los

In [5]:
cfg.log_config.hooks[1].init_kwargs.project = project_name
cfg.log_config.hooks[1].init_kwargs.name = wandb.run.name
cfg.runner.max_epochs 2 4

In [6]:
# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

# num_workers?
cfg.data.samples_per_gpu = 4

# seed
cfg.seed=2020

# ??
cfg.gpu_ids = [0]

# 새롭게 만든 모델을 저장할 장소
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

# class 개수 11개로 수정
cfg.model.roi_head.bbox_head.num_classes = 11

# ??
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [7]:
model = build_detector(cfg.model)

2021-05-17 08:46:52,612 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-17 08:46:52,614 - mmdet - INFO - Use load_from_torchvision loader
2021-05-17 08:46:52,866 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [8]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.43s)
creating index...
index created!


In [ ]:

# # cfg.checkpoint_config

# wandb.login()
# cfg.log_config.hooks = [
#     dict(type='TextLoggerHook'),
#     dict(
#         type='WandbLoggerHook',
#         init_kwargs=dict(
#             project='p32',
#             name=save_file_name
#         ),
#         with_step=False # 아우 자꾸 저장 해대 ㅡㅡ
#     )
# ]


In [9]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-17 08:47:07,043 - mmdet - INFO - Start running, host: root@a868eb526110, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/faster_rcnn_r50_fpn_1x_trash
2021-05-17 08:47:07,047 - mmdet - INFO - workflow: [('train', 1)], max: 2 epochs
Done (t=1.16s)
creating index...
index created!
/opt/ml/code/mmdetection_trash/mmdet/core/evaluation/eval_hooks.py:113: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')


2021-05-17 08:47:33,879 - mmdet - INFO - Epoch [1][50/655]	lr: 1.978e-03, eta: 0:07:34, time: 0.360, data_time: 0.049, memory: 2100, loss_rpn_cls: 0.5602, loss_rpn_bbox: 0.0997, loss_cls: 0.7469, acc: 86.1191, loss_bbox: 0.1906, loss: 1.5975, grad_norm: 6.7101
2021-05-17 08:47:49,879 - mmdet - INFO - Epoch [1][100/655]	lr: 3.976e-03, eta: 0:06:51, time: 0.320, data_time: 0.006, memory: 2100, loss_rpn_cls: 0.2254, loss_rpn_bbox: 0.0813, loss_cls: 0.4346, acc: 90.3828, loss_bbox: 0.3500, loss: 1.0912, grad_norm: 2.5466
2021-05-17 08:48:05,624 - mmdet - INFO - Epoch [1][150/655]	lr: 5.974e-03, eta: 0:06:24, time: 0.315, data_time: 0.006, memory: 2100, loss_rpn_cls: 0.1678, loss_rpn_bbox: 0.0729, loss_cls: 0.4222, acc: 90.4180, loss_bbox: 0.3595, loss: 1.0223, grad_norm: 2.9491
2021-05-17 08:48:21,235 - mmdet - INFO - Epoch [1][200/655]	lr: 7.972e-03, eta: 0:06:02, time: 0.312, data_time: 0.006, memory: 2100, loss_rpn_cls: 0.1666, loss_rpn_bbox: 0.0827, loss_cls: 0.4629, acc: 88.8135, loss

train/loss_rpn_cls,0.09311
train/loss_rpn_bbox,0.07209
train/loss_cls,0.40912
train/acc,88.74023
train/loss_bbox,0.35825
train/loss,0.93257
train/grad_norm,1.99701
learning_rate,0.02
momentum,0.9
global_step,1311
_runtime,502


train/loss_rpn_cls,█▃▂▂▂▂▂▂▃▂▂▂▂▁▁▁▁▂▂▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▅▃▅▃▄▄▄▆▅▅▄▃▄▃▄▃▂▃▁▂▁▂▁▃▃
train/loss_cls,█▂▁▂▂▂▂▂▃▃▃▂▂▂▂▂▁▁▂▂▂▂▁▁▂▁
train/acc,▁██▅▆▆▅▆▆▄▅▆▅▅▅▅▅▆▆▄▄▃▆▆▅▅
train/loss_bbox,▁▆▆█▇▇█▇▇█▇▇▇▇▇▇▇▆▆▇▇▇▆▆▆▆
train/loss,█▃▂▃▃▃▃▃▄▃▃▂▃▂▂▂▂▂▂▂▂▂▁▁▂▁
train/grad_norm,█▂▂▃▃▃▂▃▃▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁
learning_rate,▁▂▃▃▄▅▆▆▇███████████████████
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇█
